In [420]:
from mne.preprocessing.nirs import optical_density, temporal_derivative_distribution_repair, scalp_coupling_index
import mne
# Other Tooling
import pandas as pd
import numpy as np
import os, warnings
from collections import defaultdict
from copy import deepcopy
import importlib
import statsmodels.formula.api as smf
from scipy import stats
from mne.preprocessing.nirs import beer_lambert_law
from mne import Epochs, events_from_annotations

In [421]:
def make_snirf_path(dir,sub_path):
    """
    Simply finds and returns the path of the snirf file in the subjects folder
    """
    for file in dir:
        target = '.snirf'
        if target in file:
            snirf_path = f'{sub_path}/{file}'
            return snirf_path

In [422]:
# Variables
root = '../fnirs/All/'
ignore = ['.DS_Store']
trigger_id = {'1': 'Visual', '2': 'Sound', '3': 'Both'}
events = pd.read_csv('./attention_shift_events.csv')


In [423]:
def add_event_triggers(events):
    all_mne_events = []
    for event in events:
        mne_events = event['mne_events']
        for item in mne_events:
            all_mne_events.append(item)
        
        # Write the events to the snirf or fif file
        mne.annotations_from_events()

In [424]:
def align_triggers_with_events(triggers, trigger_dict, sub_events):
    trigger_map = {1: 'Both', 2: 'Sound', 3: 'Visual'}
    aligned_events = []
    count = 0
    for id, trigger in enumerate(triggers):
        data = {}
        event_type = trigger_map[trigger[-1]]
        mask = (sub_events['condition'] == event_type) & (sub_events['trial_id'] == count)
        conditional_events = sub_events[mask]

        # Add items to data dict
        data['trigger'] = trigger
        data['raw_events'] = conditional_events

        aligned_events.append(data)

        # Delicatly Increment count
        currrent_trigger = trigger[-1]
        next_trigger = triggers[id+1][-1] if id < len(triggers)-1 else triggers[-1][-1]

        count = count + 1 if currrent_trigger == next_trigger else 0

    return aligned_events

In [425]:
def convert_events_to_MNE_format(events, sampling_rate):
    # Takes in an array of dicts containing trigger and event data and formats the events
    # to better fit the MNE model for triggers
    updated_events = []
    for event in events:
        mne_events = []
        # print(event)
        raw_events = event['raw_events']
        trigger_sample = event['trigger'][0]
        # interate through the events df
        for index, row in raw_events.iterrows():
            seconds_after_trigger = row['seconds_after_trigger']
            # convert the time difference into sample differences for MNE
            event_sample = round(trigger_sample + (seconds_after_trigger * sampling_rate))
            mne_events.append([event_sample, 0, 4])
            
            # Save the event to the dict and create new events array
            event['mne_events'] = mne_events
        updated_events.append(event)
    return updated_events

In [426]:

def get_raw_signal(path):
    subs = os.listdir(path)

    for sub in subs[:2]:
        if sub not in ignore:
            sub_id = int(sub.split('-')[-1])
            sub_path = f'{root}/{sub}/nirs'
            sub_nirs_directory = os.listdir(sub_path)
            snirf_path = make_snirf_path(sub_nirs_directory, sub_path)

            # Generate Raw OD values
            raw_intensity = mne.io.read_raw_snirf(snirf_path, verbose=True, preload=False)
            # Resample if needed
            # raw_intensity.load_data().resample(4.0, npad="auto")
            raw_intensity.annotations.rename(trigger_id)
            raw_od = optical_density(raw_intensity)
            raw_haemo = beer_lambert_law(raw_od, ppf=0.1)

            # Find the sampling rate
            sampling_rate = raw_haemo.info['sfreq']

            # Converts the snirf data into a CSV
            # data = pd.DataFrame(raw_haemo.get_data())
            # data.to_csv('hemo_data.csv')

            # Get list of events
            triggers, trigger_dict = events_from_annotations(raw_haemo, verbose=False)
            sub_events = events[events['sub_id'] == sub_id]
            sub_events.to_csv('sub_events.csv')
            
            # Pair the events with their proper timestamps for SVM and waveform plotting.
            paired_events = align_triggers_with_events(triggers, trigger_dict, sub_events)

            # Convert the raw_events into an MNE event data type of [sample, 0, type]
            # print(len(paired_events))
            formated_paired_events = convert_events_to_MNE_format(paired_events, sampling_rate)

            # Write the events into the snirf file as a trigger
            add_event_triggers(formated_paired_events)
            
            



In [427]:
get_raw_signal(root)

Loading /Users/nolanbrady/Desktop/AttentionShiftingData/scripts/../fnirs/All/sub-6/nirs/2023-05-25_001.snirf
Reading 0 ... 14825  =      0.000 ...  1457.297 secs...
[[429, 0, 4], [585, 0, 4], [891, 0, 4]]
[[1242, 0, 4], [1353, 0, 4], [1511, 0, 4], [1673, 0, 4]]
[[2058, 0, 4], [2165, 0, 4], [2312, 0, 4], [2404, 0, 4], [2507, 0, 4], [2562, 0, 4]]
[[2913, 0, 4], [3001, 0, 4], [3123, 0, 4], [3268, 0, 4], [3360, 0, 4], [3512, 0, 4]]
[[6031, 0, 4], [6167, 0, 4], [6241, 0, 4], [6330, 0, 4], [6513, 0, 4], [6605, 0, 4]]
[[6844, 0, 4], [6926, 0, 4], [7000, 0, 4], [7057, 0, 4], [7134, 0, 4], [7203, 0, 4], [7273, 0, 4]]
[[7663, 0, 4], [7734, 0, 4], [7805, 0, 4], [7896, 0, 4], [8016, 0, 4], [8091, 0, 4], [8161, 0, 4]]
[[9006, 0, 4], [9090, 0, 4], [9168, 0, 4], [9236, 0, 4], [9307, 0, 4], [9388, 0, 4], [9455, 0, 4]]
[[11027, 0, 4], [11106, 0, 4], [11186, 0, 4], [11345, 0, 4], [11411, 0, 4], [11478, 0, 4]]
[[11840, 0, 4], [11908, 0, 4], [11984, 0, 4], [12056, 0, 4], [12135, 0, 4], [12214, 0, 4], [122